# Training

# Import packages

In [1]:
import pandas as pd
from pathlib import Path
import numpy as np
import ast

from lightning import pytorch as pl
from lightning.pytorch.callbacks import ModelCheckpoint

from chemprop import data, featurizers, models, nn
from chemprop.utils import make_mol

# Change data inputs here

In [2]:
chemprop_dir = Path.cwd().parent
input_path = chemprop_dir / "tests" / "data" / "atomic_regression_atom_mapped_input_copy.csv" # path to your data .csv file
num_workers = 0 # number of workers for dataloader. 0 means using main process for data loading
smiles_column = 'smiles' # name of the column containing SMILES strings
target_columns = ['charges', 'charges2'] # list of names of the columns containing targets

## Load data

In [3]:
df_input = pd.read_csv(input_path)
df_input

,smiles,charges,charges2
0,[S:1]1[S:2][C:3]2([C:23]([H:41])([H:42])[H:43]...,"[-0.093451, -0.121894, 0.159898, -0.198989, 0....","[-0.093451, -0.121894, 0.159898, -0.198989, 0...."
1,[Cl:1][c:2]1[c:3]([C:9]([n:10]2[c:11](=[O:12])...,"[-0.072057, 0.128813, -0.014234, -0.095166, -0...","[-0.072057, 0.128813, -0.014234, -0.095166, -0..."
2,[C:1]([C:2](=[O:3])[N:4]1[C:5]([H:21])([H:22])...,"[-0.47105, 0.575086, -0.557499, -0.171529, -0....","[-0.47105, 0.575086, -0.557499, -0.171529, -0...."
3,[C:1]([N:2]([C:3]([H:21])([H:22])[H:23])[c:4]1...,"[-0.220661, -0.131327, -0.220029, 0.338192, -0...","[-0.220661, -0.131327, -0.220029, 0.338192, -0..."
4,[C:1]([N:2]([C:3]([C:4]([C:5]([H:30])([H:31])[...,"[-0.206838, -0.184143, -0.07144, 0.180829, -0....","[-0.206838, -0.184143, -0.07144, 0.180829, -0...."
...,...,...,...
495,[C:1]([c:2]1[n:3][c:4]([N:5]([C:6]([H:29])([H:...,"[-0.399796, 0.334556, -0.490008, 0.462631, -0....","[-0.399796, 0.334556, -0.490008, 0.462631, -0...."
496,[Cl:1][C:2]1([H:17])[C:3]([O:13][C:14]([C:15](...,"[-0.194226, -0.004708, 0.142631, -0.170452, 0....","[-0.194226, -0.004708, 0.142631, -0.170452, 0...."
497,[S:1]=[C:2]([N:3]1[C:4]([H:21])([H:22])[C:5]([...,"[-0.519424, 0.238167, -0.04533, -0.064195, 0.0...","[-0.519424, 0.238167, -0.04533, -0.064195, 0.0..."
498,[C:1]([C:2]([C:3]([H:24])([H:25])[H:26])([C:4]...,"[-0.3687, 0.12996, -0.366706, -0.152338, -0.19...","[-0.3687, 0.12996, -0.366706, -0.152338, -0.19..."


## Get SMILES and targets

In [4]:
smis = df_input.loc[:, smiles_column].values
ys = df_input.loc[:, target_columns]

In [5]:
smis[:2] # show first 2 SMILES strings

array(['[S:1]1[S:2][C:3]2([C:23]([H:41])([H:42])[H:43])[N:4]([C:20]([C:21]([C:22]([H:38])([H:39])[H:40])([H:36])[H:37])([H:34])[H:35])[C:5](=[O:6])[C:7]13[N:8]([c:9]1[c:10]([c:11]([H:24])[c:12]([H:25])[c:13]([H:26])[c:14]1[H:27])[C:15]3([C:16]([H:28])([H:29])[H:30])[C:17]([H:31])([H:32])[H:33])[C:18]2=[O:19]',
       '[Cl:1][c:2]1[c:3]([C:9]([n:10]2[c:11](=[O:12])[c:13]([N+:14]([O-:15])=[O:16])[c:17]([H:25])[c:18]([H:26])[c:19]2[H:27])([H:23])[H:24])[c:4]([H:20])[c:5]([H:21])[c:6]([F:7])[c:8]1[H:22]'],
      dtype=object)

In [6]:
ys[:5] # show first 5 molecule targets

,charges,charges2
0,"[-0.093451, -0.121894, 0.159898, -0.198989, 0....","[-0.093451, -0.121894, 0.159898, -0.198989, 0...."
1,"[-0.072057, 0.128813, -0.014234, -0.095166, -0...","[-0.072057, 0.128813, -0.014234, -0.095166, -0..."
2,"[-0.47105, 0.575086, -0.557499, -0.171529, -0....","[-0.47105, 0.575086, -0.557499, -0.171529, -0...."
3,"[-0.220661, -0.131327, -0.220029, 0.338192, -0...","[-0.220661, -0.131327, -0.220029, 0.338192, -0..."
4,"[-0.206838, -0.184143, -0.07144, 0.180829, -0....","[-0.206838, -0.184143, -0.07144, 0.180829, -0...."


In [7]:
Y = []
for molecule in range(len(ys)):
    list_props = []
    for prop in target_columns:
        np_prop = np.array(ast.literal_eval(ys.iloc[molecule][prop]))
        np_prop = np.expand_dims(np_prop, axis=1)
        list_props.append(np_prop)
    Y.append(np.hstack(list_props))

## Get molecule datapoints

In [8]:
all_data = [data.MoleculeDatapoint.from_smi(smi, y, keep_h = True) for smi, y in zip(smis, Y)]

## Perform data splitting for training, validation, and testing

In [9]:
# available split types
list(data.SplitType.keys())

['SCAFFOLD_BALANCED',
 'RANDOM_WITH_REPEATED_SMILES',
 'RANDOM',
 'KENNARD_STONE',
 'KMEANS']

In [10]:
mols = [d.mol for d in all_data]  # RDkit Mol objects are use for structure based splits
train_indices, val_indices, test_indices = data.make_split_indices(mols, "random", (0.8, 0.1, 0.1))
train_data, val_data, test_data = data.split_data_by_indices(
    all_data, train_indices, val_indices, test_indices
)

## Get AtomDataset

In [11]:
featurizer = featurizers.SimpleMoleculeMolGraphFeaturizer()

train_dset = data.AtomDataset(train_data[0], featurizer)
scaler = train_dset.normalize_targets()

val_dset = data.AtomDataset(val_data[0], featurizer)
val_dset.normalize_targets(scaler)

test_dset = data.AtomDataset(test_data[0], featurizer)

all_dset = data.AtomDataset(all_data, featurizer)
slices = all_dset._slices

## Get DataLoader

In [12]:
train_loader = data.build_dataloader(train_dset, num_workers=num_workers)
val_loader = data.build_dataloader(val_dset, num_workers=num_workers, shuffle=False)
test_loader = data.build_dataloader(test_dset, num_workers=num_workers, shuffle=False)
all_loader = data.build_dataloader(all_dset, num_workers=num_workers, shuffle=False)

# Change Message-Passing Neural Network (MPNN) inputs here

## Message Passing
A `Message passing` constructs molecular graphs using message passing to learn node-level hidden representations.

Options are `mp = nn.BondMessagePassing()` or `mp = nn.AtomMessagePassing()`

In [13]:
mp = nn.BondMessagePassing() #include why aggregation isn't used

## Feed-Forward Network (FFN)

A `FFN` takes the aggregated representations and make target predictions.

Available options can be found in `nn.PredictorRegistry`.

For regression:
- `ffn = nn.RegressionFFN()`
- `ffn = nn.MveFFN()`
- `ffn = nn.EvidentialFFN()`

For classification:
- `ffn = nn.BinaryClassificationFFN()`
- `ffn = nn.BinaryDirichletFFN()`
- `ffn = nn.MulticlassClassificationFFN()`
- `ffn = nn.MulticlassDirichletFFN()`

For spectral:
- `ffn = nn.SpectralFFN()` # will be available in future version

In [14]:
print(nn.PredictorRegistry)

ClassRegistry {
    'regression': <class 'chemprop.nn.predictors.RegressionFFN'>,
    'regression-mve': <class 'chemprop.nn.predictors.MveFFN'>,
    'regression-evidential': <class 'chemprop.nn.predictors.EvidentialFFN'>,
    'regression-quantile': <class 'chemprop.nn.predictors.QuantileFFN'>,
    'classification': <class 'chemprop.nn.predictors.BinaryClassificationFFN'>,
    'classification-dirichlet': <class 'chemprop.nn.predictors.BinaryDirichletFFN'>,
    'multiclass': <class 'chemprop.nn.predictors.MulticlassClassificationFFN'>,
    'multiclass-dirichlet': <class 'chemprop.nn.predictors.MulticlassDirichletFFN'>,
    'spectral': <class 'chemprop.nn.predictors.SpectralFFN'>
}


In [15]:
output_transform = nn.UnscaleTransform.from_standard_scaler(scaler)

In [16]:
ffn = nn.RegressionFFN(output_transform=output_transform, n_tasks=2)

## Batch Norm
A `Batch Norm` normalizes the outputs of the aggregation by re-centering and re-scaling.

Whether to use batch norm

In [17]:
batch_norm = True

## Metrics
`Metrics` are the ways to evaluate the performance of model predictions.

Available options can be found in `metrics.MetricRegistry`, including

In [18]:
print(nn.metrics.MetricRegistry)

ClassRegistry {
    'mse': <class 'chemprop.nn.metrics.MSE'>,
    'mae': <class 'chemprop.nn.metrics.MAE'>,
    'rmse': <class 'chemprop.nn.metrics.RMSE'>,
    'bounded-mse': <class 'chemprop.nn.metrics.BoundedMSE'>,
    'bounded-mae': <class 'chemprop.nn.metrics.BoundedMAE'>,
    'bounded-rmse': <class 'chemprop.nn.metrics.BoundedRMSE'>,
    'r2': <class 'chemprop.nn.metrics.R2Score'>,
    'binary-mcc': <class 'chemprop.nn.metrics.BinaryMCCMetric'>,
    'multiclass-mcc': <class 'chemprop.nn.metrics.MulticlassMCCMetric'>,
    'roc': <class 'chemprop.nn.metrics.BinaryAUROC'>,
    'prc': <class 'chemprop.nn.metrics.BinaryAUPRC'>,
    'accuracy': <class 'chemprop.nn.metrics.BinaryAccuracy'>,
    'f1': <class 'chemprop.nn.metrics.BinaryF1Score'>
}


In [19]:
metric_list = [nn.metrics.RMSE(), nn.metrics.MAE()] # Only the first metric is used for training and early stopping

## Constructs MPNN

In [20]:
agg = nn.NoAggregation()
atom_mpnn = models.MPNN(mp, agg, ffn, batch_norm, metric_list)

atom_mpnn

MPNN(
  (message_passing): BondMessagePassing(
    (W_i): Linear(in_features=86, out_features=300, bias=False)
    (W_h): Linear(in_features=300, out_features=300, bias=False)
    (W_o): Linear(in_features=372, out_features=300, bias=True)
    (W_o_b): Linear(in_features=314, out_features=300, bias=True)
    (dropout): Dropout(p=0.0, inplace=False)
    (tau): ReLU()
    (V_d_transform): Identity()
    (E_d_transform): Identity()
    (graph_transform): Identity()
  )
  (agg): NoAggregation()
  (bn): BatchNorm1d(300, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (predictor): RegressionFFN(
    (ffn): MLP(
      (0): Sequential(
        (0): Linear(in_features=300, out_features=300, bias=True)
      )
      (1): Sequential(
        (0): ReLU()
        (1): Dropout(p=0.0, inplace=False)
        (2): Linear(in_features=300, out_features=2, bias=True)
      )
    )
    (criterion): MSE(task_weights=[[1.0, 1.0]])
    (output_transform): UnscaleTransform()
  )
  (X_d_transf

# Set up trainer

In [21]:
# Configure model checkpointing
checkpointing = ModelCheckpoint(
    "checkpoints",  # Directory where model checkpoints will be saved
    "best-{epoch}-{val_loss:.2f}",  # Filename format for checkpoints, including epoch and validation loss
    "val_loss",  # Metric used to select the best checkpoint (based on validation loss)
    mode="min",  # Save the checkpoint with the lowest validation loss (minimization objective)
    save_last=True,  # Always save the most recent checkpoint, even if it's not the best
)

trainer = pl.Trainer(
    logger=False,
    enable_checkpointing=True, # Use `True` if you want to save model checkpoints. The checkpoints will be saved in the `checkpoints` folder.
    enable_progress_bar=True,
    accelerator="auto",
    devices=1,
    max_epochs=50, # number of epochs to train for
    overfit_batches=1.00,
    callbacks=[checkpointing],
)

GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
`Trainer(overfit_batches=1.0)` was configured so 100% of the batches will be used..


# Start training

In [22]:
trainer.fit(atom_mpnn, train_loader, val_loader)

/opt/anaconda3/envs/chemprop/lib/python3.11/site-packages/lightning/pytorch/callbacks/model_checkpoint.py:654: Checkpoint directory /Users/brianli/Documents/chemprop/examples/checkpoints exists and is not empty.
Loading `train_dataloader` to estimate number of stepping batches.
/opt/anaconda3/envs/chemprop/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:251: You requested to overfit but enabled train dataloader shuffling. We are turning off the train dataloader shuffling for you.
/opt/anaconda3/envs/chemprop/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:424: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.

  | Name            | Type               | Params | Mode 
---------------------------------------------------------------
0 | message_passing | BondMessagePass

Sanity Checking DataLoader 0:   0%|                       | 0/1 [00:00<?, ?it/s]

/opt/anaconda3/envs/chemprop/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:424: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.
/Users/brianli/Documents/chemprop/chemprop/nn/message_passing/base.py:271: UserWarning: The operator 'aten::scatter_reduce.two_out' is not currently supported on the MPS backend and will fall back to run on the CPU. This may have performance implications. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/mps/MPSFallback.mm:13.)
  M_all = torch.zeros(len(bmg.V), H.shape[1], dtype=H.dtype, device=H.device).scatter_reduce_(


Epoch 0: 100%|█████████████| 7/7 [00:01<00:00,  5.60it/s, train_loss_step=0.951]
Validation: |                                             | 0/? [00:00<?, ?it/s]
Validation DataLoader 0: 100%|████████████████████| 1/1 [00:00<00:00, 18.47it/s]
Epoch 1: 100%|█| 7/7 [00:00<00:00, 12.03it/s, train_loss_step=0.929, val_loss=1.
Validation: |                                             | 0/? [00:00<?, ?it/s]
Validation DataLoader 0: 100%|████████████████████| 1/1 [00:00<00:00, 16.86it/s]
Epoch 2: 100%|█| 7/7 [00:00<00:00, 12.42it/s, train_loss_step=0.924, val_loss=1.
Validation: |                                             | 0/? [00:00<?, ?it/s]
Validation DataLoader 0: 100%|████████████████████| 1/1 [00:00<00:00, 19.82it/s]
Epoch 3: 100%|█| 7/7 [00:00<00:00, 12.97it/s, train_loss_step=0.900, val_loss=1.
Validation: |                                             | 0/? [00:00<?, ?it/s]
Validation DataLoader 0: 100%|████████████████████| 1/1 [00:00<00:00, 18.43it/s]
Epoch 4: 100%|█| 7/7 [00:00<

`Trainer.fit` stopped: `max_epochs=50` reached.


Epoch 49: 100%|█| 7/7 [00:00<00:00,  9.83it/s, train_loss_step=0.467, val_loss=1


# Test results

In [23]:
results = trainer.test(atom_mpnn, test_loader)

/opt/anaconda3/envs/chemprop/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:424: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.


Testing DataLoader 0: 100%|███████████████████████| 1/1 [00:00<00:00,  9.14it/s]


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test/mae          │    0.1905066817998886     │
│         test/rmse         │    0.26353687047958374    │
└───────────────────────────┴───────────────────────────┘

# Predictions

In [24]:
from chemprop.models import load_model
import torch

individual_preds = []
model = load_model(checkpointing.best_model_path, False)
trainer = pl.Trainer(
    logger=False,
    enable_progress_bar=True,
    accelerator="auto",
    devices=1,
)

predss = trainer.predict(model, train_loader)
individual_preds.append(torch.concat(predss, 0))

average_preds = torch.mean(torch.stack(individual_preds).float(), dim=0)
test_path = chemprop_dir / "tests" / "data" / "atomic_regression_atom_mapped_input_copy.csv"
df_test = pd.read_csv(test_path, header="infer", index_col=False)

/Users/brianli/Documents/chemprop/chemprop/models/model.py:242: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  d = torch.load(path, map_location)
GPU available: True (mps), u

Predicting DataLoader 0: 100%|████████████████████| 7/7 [00:00<00:00, 12.05it/s]


## Loaded Model

In [25]:
model

MPNN(
  (message_passing): BondMessagePassing(
    (W_i): Linear(in_features=86, out_features=300, bias=False)
    (W_h): Linear(in_features=300, out_features=300, bias=False)
    (W_o): Linear(in_features=372, out_features=300, bias=True)
    (W_o_b): Linear(in_features=314, out_features=300, bias=True)
    (dropout): Dropout(p=0.0, inplace=False)
    (tau): ReLU()
    (V_d_transform): Identity()
    (E_d_transform): Identity()
    (graph_transform): Identity()
  )
  (agg): NoAggregation()
  (bn): BatchNorm1d(300, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (predictor): RegressionFFN(
    (ffn): MLP(
      (0): Sequential(
        (0): Linear(in_features=300, out_features=300, bias=True)
      )
      (1): Sequential(
        (0): ReLU()
        (1): Dropout(p=0.0, inplace=False)
        (2): Linear(in_features=300, out_features=2, bias=True)
      )
    )
    (criterion): MSE(task_weights=[[1.0, 1.0]])
    (output_transform): UnscaleTransform()
  )
  (X_d_transf

In [26]:
for i in range(len(df_test)):
    first_atom = slices.index(i)
    last_atom = first_atom + slices.count(i)
    mol_avg_preds = average_preds[first_atom:last_atom]
    df_test.loc[i, target_columns] = [str(mol_avg_preds[:,j].tolist()) for j in range(len(target_columns))]

#torch.split(average_preds, split_size_or_sections=torch.bincount(torch.tensor(slices)).tolist(), dim=0)

output_path = chemprop_dir / "tests" / "data" / "atomic_regression_atom_mapped_output.csv"
if output_path.suffix == ".pkl":
    df_test = df_test.reset_index(drop=True)
    df_test.to_pickle(output_path)
else:
    df_test.to_csv(output_path, index=False)

df_test

,smiles,charges,charges2
0,[S:1]1[S:2][C:3]2([C:23]([H:41])([H:42])[H:43]...,"[-0.0415063202381134, 0.027678905054926872, -0...","[-0.04111367464065552, 0.049296312034130096, -..."
1,[Cl:1][c:2]1[c:3]([C:9]([n:10]2[c:11](=[O:12])...,"[0.027436845004558563, -0.004456440918147564, ...","[0.012210391461849213, -0.002815687097609043, ..."
2,[C:1]([C:2](=[O:3])[N:4]1[C:5]([H:21])([H:22])...,"[-0.032318443059921265, 0.05127551779150963, 0...","[-0.014118767343461514, 0.03646818548440933, 0..."
3,[C:1]([N:2]([C:3]([H:21])([H:22])[H:23])[c:4]1...,"[-0.049445878714323044, 0.05141334980726242, 0...","[-0.05384434014558792, 0.02201269194483757, 0...."
4,[C:1]([N:2]([C:3]([C:4]([C:5]([H:30])([H:31])[...,"[0.05141334980726242, 0.05141334980726242, 0.0...","[0.02201269194483757, 0.02201269194483757, 0.0..."
...,...,...,...
495,[C:1]([c:2]1[n:3][c:4]([N:5]([C:6]([H:29])([H:...,[],[]
496,[Cl:1][C:2]1([H:17])[C:3]([O:13][C:14]([C:15](...,[],[]
497,[S:1]=[C:2]([N:3]1[C:4]([H:21])([H:22])[C:5]([...,[],[]
498,[C:1]([C:2]([C:3]([H:24])([H:25])[H:26])([C:4]...,[],[]
